In [11]:
############################################################
# Training Classifier with vectorized data and running tests
############################################################

In [ ]:
# restarting the kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
# optionally run preprocessing again
#%run ./import_and_preprocess.ipynb
# optionally run vectorization again
#%run ./vectorize.ipynb

In [2]:
# imports
from ClassificationInterpreter import ClassificationInterpreter
from Storage import Storage
from SessionConfigReader import SessionConfigReader
from Classifier import Classifier
from EvaluationHandler import EvaluationHandler

Using TensorFlow backend.


In [3]:
vec_model_id = SessionConfigReader.read_value('vec_model_id')
nn_model_id = SessionConfigReader.read_value('keras_nn_model_id')
print(vec_model_id)
print(nn_model_id)
train_vec_id = 'articles_train_vectorized'
test_vec_id = 'articles_test_vectorized'

articles_train_word2vec_001
articles_train_keras_nn_001


In [4]:
Storage.delete_h5_model(nn_model_id)

In [5]:
train_vec = Storage.load_pd_frame(train_vec_id)
test_vec = Storage.load_pd_frame(test_vec_id)

In [6]:
train_vec

,categories,text,noise removed,stopwords removed,preprocessed,document vector
9254,[Wirtschaft],Der amerikanische Rohstoffriese Samson Resourc...,Der amerikanische Rohstoffriese Samson Resourc...,amerikanische Rohstoffriese Samson Resources p...,amerikanische Rohstoffriese Samson Resources p...,"[0.35116067564275155, 0.10396467288736985, -0...."
3584,[Kultur],"Wenn man den Menschen in Glück ersäuft, dann f...",Wenn man den Menschen in Glück ersäuft dann fä...,Menschen Glück ersäuft fällt Glücklichen Besse...,Mensch glücken ersaufen fällen Glückliche Bess...,"[0.27171568838982946, 0.1320875390730281, 0.04..."
7766,[Web],"Nächster Fall von Hassposting, der zu Jobverlu...",Nächster Fall von Hassposting der zu Jobverlus...,Nächster Fall Hassposting Jobverlust führt ein...,Nächster Fall Hassposting Jobverlust führen ei...,"[0.2786971673381049, 0.11880292560439557, 0.03..."
9632,[Wirtschaft],"Spaniens Energieriesen verfeuern Braunkohle, u...",Spaniens Energieriesen verfeuern Braunkohle um...,Spaniens Energieriesen verfeuern Braunkohle Be...,Spanien Energieriesen verfeuern Braunkohle Bed...,"[0.35139737237285784, 0.23918282515932243, -0...."
8732,[Wirtschaft],Ursache muss noch untersucht werden. Wien – Ku...,Ursache muss noch untersucht werden Wien Kurz ...,Ursache untersucht Wien Kurz Heiligen Abend mü...,Ursache untersuchen Wien Kurz Heilige Abend mü...,"[0.23216174450306887, 0.07704747765426706, 0.0..."
...,...,...,...,...,...,...
3029,[International],74-Jährigem drohten 350 Peitschenhiebe. London...,Jährigem drohten Peitschenhiebe London Ein Ja...,Jährigem drohten Peitschenhiebe London Jahre a...,Jährigem drohen Peitschenhiebe London Jahr alt...,"[0.356084035609329, 0.11922967937933335, -0.10..."
711,[Inland],Bernd Saurer war Bridge-Juniorenweltmeister un...,Bernd Saurer war Bridge Juniorenweltmeister un...,Bernd Saurer Bridge Juniorenweltmeister Krauss...,Bernd Saurer Bridge Juniorenweltmeister Krauss...,"[0.2291977620445826, 0.18525625636647555, 0.00..."
5772,[Sport],Finalfluch blieb Klopp treu – Sevilla unterstr...,Finalfluch blieb Klopp treu Sevilla unterstric...,Finalfluch blieb Klopp treu Sevilla unterstric...,Finalfluch bleiben Klopp treu Sevilla unterstr...,"[0.3283075856355329, 0.3772200811120759, -0.01..."
7926,[Web],Außenamt: Beurteilung durch UN-Arbeitsgruppe e...,Außenamt Beurteilung durch UN Arbeitsgruppe en...,Außenamt Beurteilung UN Arbeitsgruppe entspric...,Außenamt Beurteilung UN Arbeitsgruppe entsprec...,"[0.2781579718575813, 0.10152218217262998, 0.00..."


In [8]:
train_vec = train_vec.drop(columns=['noise removed', 'stopwords removed', 'preprocessed'])
train_vec_cats_out = ClassificationInterpreter.create_out_vectors(train_vec)
train_vec_cats_out

,categories,text,document vector,category vector
9254,[Wirtschaft],Der amerikanische Rohstoffriese Samson Resourc...,"[0.35116067564275155, 0.10396467288736985, -0....","[0.6316933482885361, 0.5150488093495369, 0.535..."
3584,[Kultur],"Wenn man den Menschen in Glück ersäuft, dann f...","[0.27171568838982946, 0.1320875390730281, 0.04...","[0.5749996975064278, 0.5365331657230854, 0.557..."
7766,[Web],"Nächster Fall von Hassposting, der zu Jobverlu...","[0.2786971673381049, 0.11880292560439557, 0.03...","[0.5425518825650215, 0.5334372371435165, 0.504..."
9632,[Wirtschaft],"Spaniens Energieriesen verfeuern Braunkohle, u...","[0.35139737237285784, 0.23918282515932243, -0....","[0.6316933482885361, 0.5150488093495369, 0.535..."
8732,[Wirtschaft],Ursache muss noch untersucht werden. Wien – Ku...,"[0.23216174450306887, 0.07704747765426706, 0.0...","[0.6316933482885361, 0.5150488093495369, 0.535..."
...,...,...,...,...
3029,[International],74-Jährigem drohten 350 Peitschenhiebe. London...,"[0.356084035609329, 0.11922967937933335, -0.10...","[0.5741550773382187, 0.5454485453665257, 0.493..."
711,[Inland],Bernd Saurer war Bridge-Juniorenweltmeister un...,"[0.2291977620445826, 0.18525625636647555, 0.00...","[0.5246094726026058, 0.5045751528814435, 0.502..."
5772,[Sport],Finalfluch blieb Klopp treu – Sevilla unterstr...,"[0.3283075856355329, 0.3772200811120759, -0.01...","[0.569934718310833, 0.5755672007799149, 0.5210..."
7926,[Web],Außenamt: Beurteilung durch UN-Arbeitsgruppe e...,"[0.2781579718575813, 0.10152218217262998, 0.00...","[0.5425518825650215, 0.5334372371435165, 0.504..."


In [9]:
test_vec = test_vec.drop(columns=['noise removed', 'stopwords removed', 'preprocessed'])
test_vec

,categories,text,document vector
1,[Etat],App sei nicht so angenommen worden wie geplant...,"[0.43275529039757593, 0.14660344502356434, -0...."
2,[Etat],"Zum Welttag der Suizidprävention ist es Zeit, ...","[0.3089651198270509, 0.14324529158572355, 0.10..."
3,[Etat],Mitarbeiter überreichten Eigentümervertretern ...,"[0.27314701859378604, 0.1375068294044624, -0.0..."
4,[Etat],Service: Jobwechsel in der Kommunikationsbranc...,"[0.5494780564777456, 0.030977784897890184, -0...."
5,[Etat],Was Sie über diese Woche wissen sollten - und ...,"[0.42542389830729616, 0.08715114129396777, -0...."
...,...,...,...
10262,[Wissenschaft],"Archäologin: ""Einige Monumente wie der Torboge...","[0.32293636752292515, 0.0826208355545532, 0.02..."
10263,[Wissenschaft],800 Wissenschafter zu großer Konferenz in Wien...,"[0.27630487958975564, 0.1332450757531085, 0.03..."
10264,[Wissenschaft],Vor seinem Untergang befand sich das Schiff de...,"[0.3142480892858591, 0.13729275871034577, -0.0..."
10270,[Wissenschaft],Die zentrale Frage des Projekts: Siedelten Ägy...,"[0.3231603068196111, 0.1403274620559791, 0.059..."


In [10]:
# Training...
new_model_id = Classifier.create_model(train_vec_cats_out, fv_col_name='document vector', cat_v_col_name='category vector')
new_model_id

Epoch 1/5
5136/5136 [==============================] - 1s 233us/step - loss: -0.9940 - accuracy: 0.3808
Epoch 2/5
5136/5136 [==============================] - 1s 211us/step - loss: -0.9970 - accuracy: 0.6657
Epoch 3/5
5136/5136 [==============================] - 1s 207us/step - loss: -0.9973 - accuracy: 0.6505
Epoch 4/5
5136/5136 [==============================] - 1s 208us/step - loss: -0.9974 - accuracy: 0.6505
Epoch 5/5
5136/5136 [==============================] - 1s 207us/step - loss: -0.9975 - accuracy: 0.6456


'articles_train_keras_nn_001'

In [11]:
# Testing...
classified_df = Classifier.classify(train_vec)
classified_df

,categories,text,document vector,classification output
9254,[Wirtschaft],Der amerikanische Rohstoffriese Samson Resourc...,"[0.35116067564275155, 0.10396467288736985, -0....","[0.011435551, 0.010366875, 0.010283, 0.0077088..."
3584,[Kultur],"Wenn man den Menschen in Glück ersäuft, dann f...","[0.27171568838982946, 0.1320875390730281, 0.04...","[0.010714682, 0.010349297, 0.010233096, 0.0086..."
7766,[Web],"Nächster Fall von Hassposting, der zu Jobverlu...","[0.2786971673381049, 0.11880292560439557, 0.03...","[0.010610302, 0.010326226, 0.010272377, 0.0087..."
9632,[Wirtschaft],"Spaniens Energieriesen verfeuern Braunkohle, u...","[0.35139737237285784, 0.23918282515932243, -0....","[0.011305227, 0.010679943, 0.01014537, 0.00786..."
8732,[Wirtschaft],Ursache muss noch untersucht werden. Wien – Ku...,"[0.23216174450306887, 0.07704747765426706, 0.0...","[0.0105674565, 0.010185702, 0.01023047, 0.0088..."
...,...,...,...,...
3029,[International],74-Jährigem drohten 350 Peitschenhiebe. London...,"[0.356084035609329, 0.11922967937933335, -0.10...","[0.011050862, 0.010653931, 0.009844647, 0.0083..."
711,[Inland],Bernd Saurer war Bridge-Juniorenweltmeister un...,"[0.2291977620445826, 0.18525625636647555, 0.00...","[0.0104243755, 0.010181858, 0.010047556, 0.009..."
5772,[Sport],Finalfluch blieb Klopp treu – Sevilla unterstr...,"[0.3283075856355329, 0.3772200811120759, -0.01...","[0.011314723, 0.011394331, 0.010211345, 0.0078..."
7926,[Web],Außenamt: Beurteilung durch UN-Arbeitsgruppe e...,"[0.2781579718575813, 0.10152218217262998, 0.00...","[0.011193218, 0.010533898, 0.010044994, 0.0081..."


In [12]:
test_v = classified_df.at[9254,'classification output']
test_v

array([0.01143555, 0.01036687, 0.010283  , 0.00770886, 0.01210971,
       0.00913784, 0.01029242, 0.00963305, 0.01111233, 0.00872978,
       0.00952943, 0.01124776, 0.01043495, 0.00808904, 0.01082644,
       0.01103076, 0.01051788, 0.00827233, 0.01028327, 0.01146432,
       0.00907118, 0.00994115, 0.00761777, 0.00970759, 0.00944596,
       0.00942555, 0.0086467 , 0.01120366, 0.01111757, 0.00844654,
       0.01119824, 0.01152241, 0.01006359, 0.01003943, 0.00946214,
       0.00534147, 0.0086298 , 0.00930904, 0.011735  , 0.01174332,
       0.00948557, 0.00813736, 0.01269431, 0.00906114, 0.00908246,
       0.00899927, 0.01041027, 0.01054102, 0.01135683, 0.01096066,
       0.01133139, 0.0097227 , 0.00740634, 0.00920182, 0.00871138,
       0.00922063, 0.01199882, 0.00895463, 0.01081946, 0.01029402,
       0.01175907, 0.00947218, 0.00969076, 0.01048937, 0.01068216,
       0.01284228, 0.01011416, 0.01213632, 0.01139848, 0.0103773 ,
       0.01103573, 0.00726161, 0.01117651, 0.01036768, 0.01005

In [13]:
interpreted = ClassificationInterpreter.interpret_output(classified_df)
interpreted

,categories,text,document vector,classification output,result
9254,[Wirtschaft],Der amerikanische Rohstoffriese Samson Resourc...,"[0.35116067564275155, 0.10396467288736985, -0....","[0.011435551, 0.010366875, 0.010283, 0.0077088...",[Sport]
3584,[Kultur],"Wenn man den Menschen in Glück ersäuft, dann f...","[0.27171568838982946, 0.1320875390730281, 0.04...","[0.010714682, 0.010349297, 0.010233096, 0.0086...",[Sport]
7766,[Web],"Nächster Fall von Hassposting, der zu Jobverlu...","[0.2786971673381049, 0.11880292560439557, 0.03...","[0.010610302, 0.010326226, 0.010272377, 0.0087...",[Sport]
9632,[Wirtschaft],"Spaniens Energieriesen verfeuern Braunkohle, u...","[0.35139737237285784, 0.23918282515932243, -0....","[0.011305227, 0.010679943, 0.01014537, 0.00786...",[Sport]
8732,[Wirtschaft],Ursache muss noch untersucht werden. Wien – Ku...,"[0.23216174450306887, 0.07704747765426706, 0.0...","[0.0105674565, 0.010185702, 0.01023047, 0.0088...",[Sport]
...,...,...,...,...,...
3029,[International],74-Jährigem drohten 350 Peitschenhiebe. London...,"[0.356084035609329, 0.11922967937933335, -0.10...","[0.011050862, 0.010653931, 0.009844647, 0.0083...",[Sport]
711,[Inland],Bernd Saurer war Bridge-Juniorenweltmeister un...,"[0.2291977620445826, 0.18525625636647555, 0.00...","[0.0104243755, 0.010181858, 0.010047556, 0.009...",[Sport]
5772,[Sport],Finalfluch blieb Klopp treu – Sevilla unterstr...,"[0.3283075856355329, 0.3772200811120759, -0.01...","[0.011314723, 0.011394331, 0.010211345, 0.0078...",[Sport]
7926,[Web],Außenamt: Beurteilung durch UN-Arbeitsgruppe e...,"[0.2781579718575813, 0.10152218217262998, 0.00...","[0.011193218, 0.010533898, 0.010044994, 0.0081...",[Sport]


In [14]:
score = ClassificationInterpreter.evaluate_output(interpreted)
score

0.1154595015576324

In [13]:
#EvaluationHandler.clear_evaluations()
EvaluationHandler.add_evaluation(score)

In [14]:
evaluations = EvaluationHandler.load_evaluations()
evaluations

,timestamp,session id,config id,score
0,2020-02-19 00:46:17,session_001,session_config_0001,0.436052
0,2020-02-19 01:27:46,session_001,session_config_0001,0.448316


In [15]:
best_score = EvaluationHandler.compare_evaluations()
best_score

,timestamp,session id,config id,score
0,2020-02-19 01:27:46,session_001,session_config_0001,0.448316
